✅ 1. 현재 시스템과 RAG의 차이점
현재 시스템

임베딩 검색: 연구 목표를 입력하면, 유사한 연구 사례를 검색
GPT 생성: 검색된 연구 사례를 기반으로 연구 목표 & 기대 효과 추천
RAG 모델로 발전시키면

검색과 생성을 더 밀접하게 연계하여 사용자의 질의에 최적화된 정보를 반환
단순히 유사한 연구를 찾는 것이 아니라, 연구 목적에 맞게 정보를 정리하여 새로운 콘텐츠 생성


✅ 2. RAG 모델로 전환하는 핵심 요소
🔹 (1) 벡터 데이터베이스(Vector DB) 활용
현재 Pandas를 사용하여 유사 연구를 찾고 있지만, 이를 Faiss, ChromaDB, Pinecone 등의 벡터 데이터베이스(Vector DB) 로 변경하면 더 빠르고 강력한 검색이 가능해집니다.

🔹 목표: 유사 연구를 보다 빠르게 검색하고, 정확도를 높이기

🔹 (2) 연구 제목과 연구 목표를 "컨텍스트" 로 활용하여 GPT가 RAG 방식으로 응답
현재는 GPT가 입력된 프로젝트 설명을 기반으로 연구 목표를 추천하지만,
RAG 방식으로 변경하면 검색된 연구 사례를 GPT 프롬프트에 포함하여 최적화된 연구 목표를 생성할 수 있습니다.

🔹 목표: GPT의 응답이 검색 결과를 적극적으로 활용하여 보다 정밀한 연구 목표 생성



In [12]:
pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.Collecting faiss-cpu
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.7 MB 1.3 MB/s eta 0:00:11
   - -------------------------------------- 0.5/13.7 MB 8.3 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/13.7 MB 11.1 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/13.7 MB 13.0 MB/s eta 0:00:01
   --------- ------------------------------ 3.2/13.7 MB 15.7 MB/s eta 0:00:01
   ------------ --------------------------- 4.4/13.7 MB 17.7 MB/s eta 0:00:01
   ---------------- ----------------------- 5.7/13.7 MB 19.3 MB/s eta 0:00:01
   ------------------ --------------------- 6.3/13.7 MB 19.0 MB/s eta 0:00:01
   ---------------------- ----------------- 7.6/13.7 MB 19.5 MB/s eta 0:00:01
   -------------------------- ------------- 9.0/13.7 MB 20.6 MB/s eta 0:00:01
   ------------------------------ --------- 10.6/13.7 MB 23.4 MB/s


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import openai
import pandas as pd
import numpy as np
import faiss  # ✅ FAISS (Facebook AI Similarity Search)
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import os
import ast  # ✅ 안전한 문자열 변환을 위한 라이브러리

# ✅ .env 파일 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ tqdm 진행률 표시 적용
tqdm.pandas()

In [2]:
# ✅ 저장된 임베딩 CSV 파일 불러오기 (기존 임베딩 데이터 활용)
df = pd.read_csv("merged_R&D_Embedding.csv", encoding="utf-8")
df["embedding"] = df["embedding"].apply(ast.literal_eval)  # 문자열을 리스트로 변환
df["embedding"] = df["embedding"].apply(np.array)  # 리스트를 numpy 배열로 변환


C:\Users\jihoo\AppData\Local\Temp\ipykernel_31352\3460159772.py:2: DtypeWarning: Columns (0,3,9,10,12,15,25,27,29,31,33,35,40,46,47,48,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged_R&D_Embedding.csv", encoding="utf-8")


In [3]:
# ✅ FAISS 벡터 데이터베이스 구축
embedding_dim = len(df["embedding"].iloc[0])  # 임베딩 차원 (1536차원)
index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 검색

# ✅ FAISS에 임베딩 추가
embedding_matrix = np.vstack(df["embedding"].values).astype('float32')  # NumPy 배열 변환
index.add(embedding_matrix)

print(f"✅ FAISS 벡터 DB에 {len(df)}개 연구 과제 임베딩이 저장되었습니다.")



✅ FAISS 벡터 DB에 50103개 연구 과제 임베딩이 저장되었습니다.


In [4]:
# ✅ 사용자의 입력을 OpenAI 임베딩으로 변환하는 함수
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.Embedding.create(input=[text], model=model)
    return np.array(response["data"][0]["embedding"], dtype='float32')


# ✅ 유사한 연구 과제 검색 함수 (FAISS 활용)
def search_similar_projects_faiss(df, project_description, top_n=10):
    """
    사용자의 연구 설명과 가장 유사한 연구 과제 n개 검색 (FAISS 활용)
    """
    print("\n🔍 FAISS 기반 유사 연구 과제 검색 중...")

    # ✅ 사용자의 입력을 임베딩 변환
    project_embedding = get_embedding(project_description).reshape(1, -1)

    # ✅ FAISS에서 가장 가까운 벡터 찾기
    distances, indices = index.search(project_embedding, top_n)

    # ✅ 검색된 연구 과제 반환
    similar_projects = df.iloc[indices[0]].copy()
    similar_projects["similarity"] = 1 / (1 + distances[0])  # 유사도 변환 (1 / (1 + L2 거리))
    
    return similar_projects


# ✅ 연구 목표 및 기대효과 생성 (GPT-4o 활용)
def generate_research_goal_effect(user_goal, user_effect, similar_projects):
    """
    GPT-4o를 활용하여 연구 목표 및 기대효과를 생성하는 함수
    """
    print("\n🎯 연구 목표 및 기대효과 추천 + 차별점 분석 중...")

    # ✅ 기존 연구 사례를 기반으로 컨텍스트 생성
    context = "\n".join([
        f"과제명: {row['과제명(국문)']}\n연구목표: {row['연구목표요약']}\n기대효과: {row['기대효과요약']}\n"
        for _, row in similar_projects.iterrows()
    ])
    
    prompt = (
        f"다음은 기존 연구 과제들의 연구 목표 및 기대효과입니다:\n{context}\n\n"
        f"📌 사용자의 연구 목표:\n{user_goal}\n"
        f"🌟 사용자의 기대 효과:\n{user_effect}\n\n"
        f"🔍 기존 연구와 비교했을 때, 사용자의 연구 목표와 기대효과에서 차별화할 수 있는 요소를 분석하여 작성해 주세요. "
        f"또한 어떤 방향으로 연구 목표 및 기대효과를 보완하면 더욱 독창적이고 효과적인 연구 기획이 될 수 있는지 제안해 주세요."
    )

    # ✅ OpenAI GPT 호출
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are an AI assistant specializing in research project planning."},
                  {"role": "user", "content": prompt}],
        max_tokens=1000,
        temperature=0.7
    )

    return response["choices"][0]["message"]["content"].strip()



In [5]:
# ✅ 사용자 입력 받기 (여러 개 입력 가능)
user_inputs = []
while True:
    user_input = input("📌 연구 목표를 입력하세요 (종료: 'exit'): ")
    if user_input.lower() == "exit":
        break
    user_inputs.append(user_input)

# ✅ 각 입력별로 유사 연구 과제 검색 및 연구 목표 생성
results = []
for user_business in user_inputs:
    print("\n📝 입력된 연구 목표: ", user_business)  

    # ✅ FAISS 기반 유사 연구 과제 검색
    similar_projects = search_similar_projects_faiss(df, user_business, top_n=10)

    # ✅ 검색된 연구 과제 결과 확인
    print("\n📌 [🔎 검색된 유사 연구 과제 TOP 10]")
    display(similar_projects[["과제명(국문)", "연구목표요약", "기대효과요약", "similarity"]])
    
    # ✅ 연구 목표 및 기대효과 생성
    recommended_goal_effect = generate_research_goal_effect(user_business, "연구 기대효과 미입력", similar_projects)

    # ✅ 연구 목표 추천 결과 출력
    print("\n📌 [✨ GPT-4o 기반 연구 목표 및 기대효과 추천]")
    print(recommended_goal_effect)

    # ✅ 결과 저장
    results.append({
        "사용자 입력 - 연구 목표": user_business,
        "추천된 연구 목표 및 기대 효과 (차별점 포함)": recommended_goal_effect,
        "유사 연구 과제": similar_projects[["과제명(국문)", "연구목표요약", "기대효과요약", "similarity"]].to_dict(orient="records")
    })

# ✅ 결과를 Pandas 데이터프레임으로 변환하여 저장
df_results = pd.DataFrame(results)
df_results.to_csv("RAG_Research_Results.csv", index=False, encoding="utf-8")

print("\n✅ 연구 목표 및 기대효과 분석 결과가 'RAG_Research_Results.csv'에 저장되었습니다.")


📝 입력된 연구 목표:  배터리 데이터를 수집하여 전기 이륜차 및 배터리에 관한 리스크를 예측, 금융 상품 개발에 활용

🔍 FAISS 기반 유사 연구 과제 검색 중...

📌 [🔎 검색된 유사 연구 과제 TOP 10]


,과제명(국문),연구목표요약,기대효과요약,similarity
24159,운행전기차LTE통신및배터리스마트관리를위한 AI기반 BaaS(Battery as a ...,2종 설치 및 시험Broker 서버 이중화 설계OBDII장치 SW 개발클라우드 플랫...,개발성과 활용 계획* 전기차에 맞는 서비스 개발- 전기차 안전진단 평가를 위한 데이...,0.513271
39361,무선기반 배터리 데이터 collector Kit 개발,1) 본 개발은 중소형 신재생에너지(풍력/태양광/하이브리드 형태) 발전 및 융합제품...,? 에너지 최적화 설계 또는 제품 설계를 위한 기반 마련 - 스마트그리드 및 신...,0.510775
24158,운행전기차LTE통신및배터리스마트관리를위한 AI기반 BaaS(Battery as a ...,... 시험_x005FBroker 서버 이중화 구현 _x005FOBDII 장치 SW...,... 기술_x005F- 중고 거래시장 활용 및 보험비 적용 연계 활용 _x005F...,0.505620
4534,전기차 배터리 팩 셀단위 임피던스 측정기 시제품 및 화재예측이 가능한 배터리 성능평...,○ 전기차 배터리 팩 BSA 셀단위 임피던스 측정기 시제품 제작하고 임피던스 기반 ...,... BMA 커넥터에 CMU 연결하여 BMU와 통신으로 팩 전체 셀단위 임피...,0.505500
27069,고장전원의 위치 및 예측이 가능한 로컬제어반과 물탱크 물 사용량의 예측이 가능한 빌...,모든 개발된 시제품을 테스트 베드를 통하여 성능 및 동작시험하는 것이 목표1. 양산...,최근에 빌딩에너지 관리시스템이 기존의 하나의 감지장치나 센서에서 보내지는 정보를 통...,0.504574
25449,재사용 가능한 배터리의 유효 수명 측정을 위한 장치 및 AI기법에 기반한 알고리즘 개발,2차년도 개발목표는 다음에 목표를 가지고 개발을 진행하고자 함. - 2차년도는 ...,... 미치기 때문에 셀 단위로 분해 가공은 경제성 확보가 어려울 것으로 보고 있음...,0.503894
9513,실운용 현장 데이터 기반 배터리 실시간 모니터링 및 전주기 운영 솔루션 개발,본 기술개발 과제의 최종목표는 배터리를 보다 안전하고 효율적으로 사용하기 위한 세계...,... 미래 구현에 기여- 정부정책 부합성 측면 산업통상자원부는 침체된 국내 ESS...,0.502626
8881,화재예방 기능을 탑재한 배터리 운송/보관/적재 시스템 개발,- BMS 데이터 연동 배터리 실시간 모니터링 시스템 개발 > BMS 데이터 획득 ...,- 배터리 전/후방산업 전체 화재예방 기여 > 배터리 화재안전 모니터링을 통한 안전...,0.502445
2101,AI 기반 배터리 슬러리 측정 및 분석 모듈 개발,"1. 최종 목표: 실험실에서 구현된 배터리 슬러리 진단 기술을 scale-up하고,...",... 판매는 진단 키트를 활용한 솔루션 제공 및 용역 형태로 이루어 짐)2. 기대...,0.501475
4886,AI기반 재사용배터리(2nd-Life Battery)의 안전성 및 효율성 강화를 위...,- 리튬이온배터리(LIB) 및 재사용배터리(2nd-Life Battery)를 적용한...,- 해외 전력소외지역(Off-grid 및 Weak-grid) 하이브리드 마이크로그리...,0.500998



🎯 연구 목표 및 기대효과 추천 + 차별점 분석 중...

📌 [✨ GPT-4o 기반 연구 목표 및 기대효과 추천]
사용자의 연구 목표는 배터리 데이터를 수집하여 전기 이륜차 및 배터리에 관한 리스크를 예측하고 이를 금융 상품 개발에 활용하는 것입니다. 이는 기존 연구들에서 다루지 않았던 금융 분야와의 융합을 주요 차별점으로 볼 수 있습니다. 다음은 사용자 연구의 차별화 요소와 보완 제안입니다.

### 차별화 요소:

1. **금융 상품 개발**:
   - 기존 연구들은 주로 기술적 개선이나 산업적 응용에 중점을 두었지만, 사용자의 연구는 배터리 데이터를 활용하여 금융 리스크를 예측하고 이를 기반으로 금융 상품을 개발한다는 점에서 차별화됩니다. 이는 보험, 금융상품 설계 등 새로운 시장을 창출할 수 있는 가능성을 제공합니다.

2. **전기 이륜차에 대한 집중**:
   - 기존 연구들은 주로 전기차나 대형 배터리 시스템에 초점을 맞추고 있습니다. 전기 이륜차는 상대적으로 덜 연구된 분야로, 이 분야에 집중하는 것은 틈새 시장을 공략할 수 있는 기회를 제공합니다.

### 보완 제안:

1. **데이터 활용의 다양성**:
   - 배터리 데이터뿐만 아니라, 운행 패턴, 사용자 데이터 등을 추가로 수집하여 리스크 예측 모델의 정확성을 높일 수 있습니다. 다양한 데이터 소스를 활용하여 보다 포괄적인 예측 모델을 개발하는 것이 중요합니다.

2. **협업 및 파트너십 구축**:
   - 금융 기관, 보험사와의 협업을 통해 실질적인 금융 상품 개발에 필요한 인사이트를 얻을 수 있습니다. 이러한 협력은 연구 결과의 상용화를 촉진할 수 있습니다.

3. **사회적 및 환경적 영향**:
   - 전기 이륜차의 리스크 관리 및 금융 상품 개발이 환경적, 사회적 영향을 어떻게 개선할 수 있는지에 대한 부분도 기대효과에 포함될 수 있습니다. 예를 들어, 배터리 재사용을 통해 환경오염을 줄이고, 금융 상품을 통해 보다 많은 사용자가 전기 이륜차를 선택하도록 유도할 수 있습니다

,과제명(국문),연구목표요약,기대효과요약,similarity
18667,CSI 미션탈출방 “CSII미션을 수행하라” 시스템 구축,1. 유니티 개발 엔진을 활용한 디지털 탈출방 (Digital escape Roo...,"1. 최근 20-30대 사이에 탈출카페, 탈출방 형식의 동적인 놀이문화 확산 기여2...",0.398147
34339,4D PC블랙박스,○ 4D PC블랙박스 녹화 소프트웨어 구현 ● 365일 24시간 사용 시간 동안 ...,... 관제 모니터링하여 외부 유출 차단● 실시간 관리자의 관리감독이 필요한 업무 ...,0.394702
16622,IoT 기반 산업용 유독가스 누출 감지 및 밸브 자동제어 시스템 개발,[밸브 제어모듈 개발] 유독가스 누출시 자체 프로세서에 의한 밸브 자동 잠금 및 데...,[밸브 제어모듈 개발] 유독가스 누출시 자체 프로세서에 의한 밸브 자동 잠금 및 데...,0.393823
43457,사출기용 가스 제거 장치,▣ 최종목표 : 제습 과정의 생략으로 사출과정 축소(원료-용융-노즐(개발 제품)-사...,1. 파급효과 - 기술적 측면: 사출성형에서는 당 제품의 기술은 플라스틱 용융성형에...,0.392053
49876,8.5톤 친환경 증기회수 유조차,▣최종목표 : 8.5톤/16톤/20톤 친환경 VAPOUR RECOVERY 유조차 개...,1. 파급효과 - 언론에 보도된 바와 같이 인명사고까지 이르는 급유시스템의 문제점을...,0.391727
20222,해외 미국/유럽/중동 주요국 현지 기술적용 POS(1종) 및 보급형/AI응용 고급형...,... 기구설계/제작/보완- 상세 기구 제작 디자인(고급형 KIOSK)▶ 시스템 하...,"... 활용분야 및 방안- 연구결과물 POS(1종), KIOSK(2종) 제품 '제품...",0.389512
13238,리스크 프로세스 기반 임베디드 SW 및 통신방송게임 융합형 시스템 개발,­ 리스크 프로세스 기반 통신방송게임 융합형 홈 플랫폼 개발­ 오픈 플랫폼 기반 임...,"MIPS 계열 또는 ARM계열의 CPU를 기반의 통신, 방송, 게임을 융합한 메인 ...",0.389456
38640,공개소스인 오픈폼(OpenFOAM)을 이용한 사용자 맞춤형 열유체 해석 프로그램 플...,... - 사용자 중심의 입/출력 환경 구축 - 사용자 업무에 최적화된 환경 제...,... - 사용자 중심의 입/출력 환경 구축 - 사용자 업무에 최적화된 환경 제...,0.389289
43458,사출기용 가스 제거 장치,▣ 최종목표 : 제습 과정의 생략으로 사출과정 축소(원료-용융-노즐(개발 제품)-사...,1. 파급효과 - 기술적 측면: 사출성형에서는 당 제품의 기술은 플라스틱 용융성형에...,0.388728
47287,휴대인터넷(WiBro)용 간섭제거 중계기 개발,광중계기를 대체할 수 있는 휴대인터넷용 간섭제거 중계기 개발,... ARPU를 3.3만원으로 가정해도 3조 3천억 규모의 시장을 형성할 것으로 ...,0.388414



🎯 연구 목표 및 기대효과 추천 + 차별점 분석 중...

📌 [✨ GPT-4o 기반 연구 목표 및 기대효과 추천]
사용자의 연구 목표 "exit"와 기대효과가 구체적으로 명시되어 있지 않기 때문에, 기존의 연구 과제들과 비교하여 차별화할 수 있는 요소와 보완할 수 있는 방향을 제안하기 위해서는 추가적인 세부 정보가 필요합니다. 그러나 일반적인 연구 기획의 차별화 및 보완 방안을 제안할 수 있습니다.

1. **차별화 요소 분석 및 제안:**

   - **기술 혁신성:** 기존 연구에서 사용된 기술이나 방법론과 어떻게 차별화될 수 있는지를 고려할 수 있습니다. 예를 들어, 최신 기술이나 혁신적인 방법론을 도입하여 기존의 제한점을 극복할 수 있는 방안을 모색할 수 있습니다.
   
   - **응용 분야의 다양성:** 기존 연구가 특정 산업이나 응용 분야에 집중되어 있다면, 사용자의 연구 목표를 다양한 산업에 적용할 수 있는 가능성을 탐색해볼 수 있습니다. 예를 들어, IoT 기술을 다양한 산업군에 맞게 모듈화하여 적용할 수 있는 방안을 고려할 수 있습니다.
   
   - **사용자 경험 향상:** 사용자 중심의 설계나 인터페이스 개선을 통해 더 나은 사용자 경험을 제공하는 방향으로 차별화할 수 있습니다. 예를 들어, 디지털 탈출방 시스템에서 사용자 인터페이스를 보다 직관적으로 개선하거나, 게임 요소를 더 풍부하게 추가하는 방법을 고려할 수 있습니다.

2. **보완 방향 제안:**

   - **구체적인 연구 목표 설정:** "exit"라는 목표는 구체적이지 않기 때문에, 연구의 목표를 명확하게 정의하는 것이 중요합니다. 연구의 범위, 대상, 그리고 해결하고자 하는 문제를 명확히 설정하여 연구의 방향성을 강화할 수 있습니다.
   
   - **기대 효과의 명확화:** 연구의 기대 효과를 보다 구체적으로 설정하여 연구의 가치와 잠재적 영향을 명확히 제시할 필요가 있습니다. 예를 들어, 특정 산업에 미치는 경제적 효과, 환경적 이점, 또는 사회적 영향 등을